<center><h1>Maltese Christian Statue (MCS) Dataset Augmentation</h1>
<h2>Matthias Bartolo</h2>

</center>

In [1]:
from time import time
import numpy as np
import os
import cv2

# Define the original and augmented dataset paths
original_dataset_path = 'MCS Dataset/'
augmented_dataset_path = 'Augmented MCS Dataset1k/'

# Ensure the augmented dataset path exists
os.makedirs(augmented_dataset_path, exist_ok=True)

# Retrieve the categories
categories = os.listdir(original_dataset_path)

# Define the number of images required per category
num_images = 1_250

# Function to apply photometric augmentations
def photometric_augment(image):
    augmented_images = []
    
    # Change saturation
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    for saturation_factor in [0.5, 1.5]:  # Desaturate and oversaturate
        hsv_adjusted = hsv.copy()
        hsv_adjusted[:, :, 1] = np.clip(hsv[:, :, 1] * saturation_factor, 0, 255)
        augmented_images.append(cv2.cvtColor(hsv_adjusted, cv2.COLOR_HSV2BGR))
    
    # Change brightness
    for brightness_factor in [-50, 50]:  # Darker and brighter
        bright = cv2.convertScaleAbs(image, alpha=1, beta=brightness_factor)
        augmented_images.append(bright)
    
    # Change contrast
    for contrast_factor in [0.5, 1.5]:  # Lower contrast and higher contrast
        contrast = cv2.convertScaleAbs(image, alpha=contrast_factor, beta=0)
        augmented_images.append(contrast)
    
    return augmented_images

# Process each category
for category in categories:
    # Create category folder in the augmented dataset
    category_path = os.path.join(original_dataset_path, category)
    augmented_category_path = os.path.join(augmented_dataset_path, category)
    os.makedirs(augmented_category_path, exist_ok=True)
    
    # List images in the category
    images = os.listdir(category_path)
    num_existing_images = len(images)
    
    # Copy original images to the augmented dataset
    for image_name in images:
        src_path = os.path.join(category_path, image_name)
        dst_path = os.path.join(augmented_category_path, image_name)
        cv2.imwrite(dst_path, cv2.imread(src_path))
    
    # Check if more images are needed
    if num_existing_images >= num_images:
        continue
    
    # Augment images to reach the required count
    images_to_generate = num_images - num_existing_images
    index = 0
    while images_to_generate > 0:
        # Cycle through existing images and augment them
        image_name = images[index % num_existing_images]
        image_path = os.path.join(category_path, image_name)
        image = cv2.imread(image_path)
        
        # Apply photometric augmentations
        augmented_images = photometric_augment(image)
        for aug_image in augmented_images:
            if images_to_generate <= 0:
                break
            aug_image_name = f"{image_name.split('.')[0]}_aug_{int(time() * 1000)}.jpg"
            cv2.imwrite(os.path.join(augmented_category_path, aug_image_name), aug_image)
            images_to_generate -= 1
        
        index += 1

print("Image augmentation completed. Augmented dataset is ready!")


Image augmentation completed. Augmented dataset is ready!
